<a href="https://colab.research.google.com/github/diegoax/ALNAE-2025/blob/main/notebooks/clase15_ALNAE_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase 15 (Viernes 23 de mayo, 2025)
---

# Factorización de Matrices


## Factorización de Matrices: Positividad y Esparsidad

Tradicionalmente, se ha utilizado la descomposición en valores singulares (SVD) para descomponer una matriz $A$ como:
$$
A = U \Sigma V^T
$$
Esto es muy útil, por ejemplo, en Análisis de Componentes Principales (PCA) como ya vimos en el curso.

Sin embargo, en presencia de restricciones como **no negatividad** de las entradas de los factores o **esparsidad**, la SVD no siempre es adecuada.

A continuación, se presentan factorizaciones alternativas de $A$ que imponen nuevas propiedades útiles:

| Tipo de factorization     | Objetivo de minimización                              | Restricciones             |
|---------------------------|--------------------------------------------------------|---------------------------|
| **Matrices no negativas** | $$\min \|A - UV\|_F^2$$                                 | $U \geq 0$, $V \geq 0$    |
| **Esparsas y no negativas** | $$\min \|A - UV\|_F^2 + \lambda \|UV\|_N$$            | $U \geq 0$, $V \geq 0$    |

### Algoritmo: Iteración Alternante

Para computar $A \approx UV$, se utiliza una **iteración alternante**:

- **Actualizar $U$ con $V$ fijo**, luego
- **Actualizar $V$ con $U$ fijo**.

Cada paso es rápido ya que el problema es lineal en el factor que se está actualizando.

Este enfoque es una extensión de la idea de SVD (donde $\Sigma$ puede ser absorbido en $U$). La factorización $UV$ también se relaciona con el **algoritmo de $k$-means**, agrupando vectores columna de $A$ en $r$ clusters.

Esto es, si la columna $k$ de $A$ está en el cluster asociado al vector $u_j$, i.e., $a_k \approx u_j$, entonces en la factorización $A\approx UV$ se tiene que la columna $k$ de $V$,

En resumen, estas factorizaciones permiten extraer estructuras interpretables y ajustadas a restricciones específicas que la SVD no cumple.


---

## Proycto: Nonnegative Matrix Factorization (NMF)

El objetivo de NMF es aproximar una matriz no negativa $A \geq 0$ mediante un producto de rango bajo:
$$
A \approx UV
$$
donde $U \geq 0$ y $V \geq 0$.

### Propósito
- **Rango bajo**: simplifica los datos.
- **No negatividad**: asegura que las cantidades tengan sentido (volumen, probabilidad, conteo,distancia, etc.), evitando cancelaciones entre signos opuestos.

### Dificultades
Cuando $A$ es simétrica definida positiva, se podría esperar encontrar una matriz $B \geq 0$ tal que $B^T B = A$, pero esto raramente ocurre (?). En su lugar, se puede buscar una matriz $B$ no negativa que minimice la diferencia con $A$ en el sentido de Frobenius:
$$
\min_{B \geq 0} \|A - B^T B\|_F
$$

### Aplicaciones
- La factorización $A \approx BC$ también se utiliza en el contexto de la reducción de dimensionalidad, compresión y visualización de datos.
- Si $C$ tiene menos columnas que $A$, se logra una representación más compacta.
- Esta técnica es útil cuando los datos son naturalmente no negativos y se desea interpretar sus componentes (por ejemplo, en imágenes, documentos, biología computacional).

### Objetivos relacionados

| Método | Factorización buscada                          |
|--------|------------------------------------------------|
| NMF    | Encontrar matrices $U$, $V$ no negativas con $A \approx UV$ |
| SPCA   | Encontrar matrices esparsas $B$, $C$ con $A \approx BC$     |

### Referencias
- Lee y Seung, *Nature* 401 (1999), 788–791.
- N. Gillis, *The Why and How of Nonnegative Matrix Factorization*, arXiv:1401.5226.
- L. Xu, B. Yu, Y. Zhang, *An alternating direction algorithm for matrix and tensor factorizations*, Computational Optimization and Applications, 68 (2017).


---

### Extracción de Rasgos Faciales

En esta aplicación, cada columna de la matriz $A$ representa una imagen facial, donde sus componentes corresponden a intensidades de píxeles, por lo que $A \geq 0$.

- El objetivo es encontrar unas pocas **"caras básicas"** contenidas en $B$, tal que:
  $$
  A \approx BC
  $$
- Estas caras básicas se combinan linealmente (sin cancelaciones negativas) para reconstruir las múltiples caras en $A$.
- Variaciones en geometría (ojos, nariz, boca) pueden ser suficientes para una buena reconstrucción.
- Esta idea está relacionada con el desarrollo de **eigenfaces** (Turk y Pentland), aunque aquí se enfatiza la **no negatividad** de los factores.

Esta factorización permite representar rostros de forma interpretable, con partes básicas que tienen sentido visual.


---

### Minería de Texto y Clasificación de Documentos

- En este contexto, cada **columna de $A$** representa un **documento**, y cada **fila** representa una **palabra**.
- La matriz $A$ suele ser esparsa y no negativa (frecuencias o TF-IDF).
- El objetivo es descomponer $A \approx BC$, donde:

  - $b_i$ es un **tema** (topic vector),
  - $c_{ij}$ es la **importancia** del tema $b_i$ en el documento $a_j$.

  $$
  a_j \approx \sum_i c_{ij} \cdot b_i
  $$

- Dado que $B \geq 0$ y $C \geq 0$, los documentos se construyen **combinando, no restando** temas.
- Esto permite una interpretación más clara: NMF **identifica temas** y **clasifica** documentos en función de ellos.

## 🧠 Ejemplo de NMF para Minería de Texto

Supongamos que tenemos 3 documentos y 4 palabras:

- **Palabras**: `["data", "science", "football", "stadium"]`
- **Documentos**: `["Doc 1", "Doc 2", "Doc 3"]`

La matriz $A$ representa la frecuencia de cada palabra en cada documento:
$$
A =
\begin{bmatrix}
2 & 0 & 1 \\\\
1 & 0 & 1 \\\\
0 & 3 & 0 \\\\
0 & 2 & 0 \\\\
\end{bmatrix}
$$
Esto significa:

- Doc 1 y Doc 3 contienen palabras de data science.
- Doc 2 contiene palabras de "football".

### 🎯 Objetivo

Aplicamos **NMF** para factorizar $A \approx BC$ con:

$$ \mbox{Documento }a_j\approx \sum_i (\mbox{imporancia }c_{i,j}) (\mbox{ tema } b_i)
$$

Luego $B$ puede pensarse como una colección de temas, y estamos clasifcando todos los documentos como combinación lineal de los temas más importantes.

## 🌿 Sparse Principal Components (SPCA)

En muchas aplicaciones (finanzas, genética, etc.) es importante tener **componentes principales dispersos**: es decir, vectores con **pocos elementos distintos de cero**. Esto permite:

- Interpretabilidad (e.g. seleccionar unas pocas variables clave).
- Evitar ruido o decisiones basadas en muchas variables con poco efecto.



### Problema con SVD tradicional

Los vectores singulares $u$ y $v$ de la SVD tienen **todos sus elementos distintos de cero**, lo que:

- Dificulta la interpretación.
- Hace que no podamos seleccionar solo unas pocas variables significativas.



### Formulación básica de SPCA

Sea $A$ una matriz de datos, (o una matriz $S$ una matriz de covarianza muestral $S=\frac{1}{n-1}X^TX$).

Una idea sería resolver el siguiente problema de optimización
$$
\max_{\|x\|=1} x^T S x - \rho \cdot \text{Card}(x)
$$

O bien:

$$
\max_{\|x\|=1} x^T S x \quad \text{sujeto a} \quad \text{Card}(x) \leq k
$$

donde:
- $S$ es la matriz de covarianza (simétrica y semi-definida positiva),
- $\text{Card}(x)$ es el número de componentes no nulos de $x$.

### Penalización con norma $\ell^1$

Como la **cardinalidad** no es eficiente para optimización, se reemplaza por una penalización $\ell^1$, que favorece la dispersión:

- Para vectores: usar $\|x\|_1$
- Para matrices: usar la **norma nuclear** $\|X\|_N$ (suma de valores singulares)



## LASSO: selección de variables

Para encontrar vectores dispersos $x$, se utiliza:

$$
\min \|Ax - b\|^2 + \lambda \sum_{i=1}^n |x_i|
$$

Esto es **LASSO**, que favorece soluciones con muchos ceros en $x$.



## Referencias para chusmear

1. R. Tibshirani (1996), *Regression shrinkage and selection via the Lasso*
2. H. Zou y T. Hastie (2005), *Regularization and variable selection via the elastic net*
3. Zou, Hastie, Tibshirani (2006), *Sparse Principal Component Analysis*
